In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import io
from PIL import Image
import os
from IPython.display import Image as IPImage
import torch
from PIL import Image as PILImage
from matplotlib.animation import FuncAnimation
from IPython.display import display, HTML
import ipywidgets as widgets
from IPython.display import display, IFrame
from matplotlib.animation import HTMLWriter
from matplotlib.patches import Circle

In [2]:
# In this version, there will be no collision of pdestrian with other pedestrians and objects.

In [3]:
class Entity:
    def __init__(self, x, y, entity_type):
        self.x = x
        self.y = y
        self.type = entity_type
        self.is_movable = entity_type != 'obstacle'
        self.prev_x = x
        self.prev_y = y

    def move(self, dx, dy):
        if self.is_movable:
            self.prev_x = self.x
            self.prev_y = self.y
            self.x += dx
            self.y += dy

In [4]:
def detect_collision(entities):
    collisions = []
    for i in range(len(entities)):
        for j in range(i+1, len(entities)):
            if entities[i].x == entities[j].x and entities[i].y == entities[j].y:
                collisions.append((entities[i], entities[j]))
    return collisions

In [5]:
class SyntheticDataGenerator:
    def __init__(self, grid_size=10, num_vehicles=5, num_pedestrians=5, num_obstacles=5):
        self.grid_size = grid_size
        self.num_vehicles = num_vehicles
        self.num_pedestrians = num_pedestrians
        self.num_obstacles = num_obstacles

    def is_valid_move(self, new_x, new_y, entities, moving_entity, next_positions=None):
        """
        Check if a move is valid based on entity type and surrounding entities.
        Rules:
        - Vehicles can collide with anything (no restrictions)
        - Pedestrians cannot collide with other pedestrians and obstacles
        - Pedestrians can collide with vehicles
        """
        # Check grid boundaries
        if new_x < 0 or new_x >= self.grid_size or new_y < 0 or new_y >= self.grid_size:
            return False

        if next_positions is None:
            next_positions = set()

        # If it's a vehicle, allow all moves within grid boundaries
        if moving_entity.type == 'vehicle':
            return True

        # For pedestrians, check collisions with obstacles and other pedestrians only
        if moving_entity.type == 'pedestrian':
            for entity in entities:
                # Check collision with obstacles
                if entity.type == 'obstacle' and entity.x == new_x and entity.y == new_y:
                    return False

                # Check collision with other pedestrians
                if entity.type == 'pedestrian' and entity != moving_entity:
                    # Check direct collision with current and predicted positions
                    if (entity.x == new_x and entity.y == new_y) or ((new_x, new_y) in next_positions):
                        return False
                    # Maintain minimum separation distance from other pedestrians
                    if abs(entity.x - new_x) + abs(entity.y - new_y) <= 1:  # Manhattan distance
                        return False

        return True

    def get_safe_move(self, entity, entities, next_positions):
        """Get a safe move direction for an entity"""
        if not entity.is_movable:
            return 0, 0

        # Define possible movements (including staying in place)
        possible_moves = [(dx, dy) for dx in [-1, 0, 1] for dy in [-1, 0, 1]]
        
        # Shuffle possible moves for randomness
        np.random.shuffle(possible_moves)
        
        if entity.type == 'pedestrian':
            # First try to find a move that's safe from both obstacles and other pedestrians
            for dx, dy in possible_moves:
                new_x = entity.x + dx
                new_y = entity.y + dy
                
                if self.is_valid_move(new_x, new_y, entities, entity, next_positions):
                    # Additional check for minimum separation from other pedestrians
                    is_safe = True
                    for other in entities:
                        if other != entity and other.type == 'pedestrian':
                            # Check if new position would be too close to another pedestrian
                            if abs(other.x - new_x) + abs(other.y - new_y) <= 1:
                                is_safe = False
                                break
                    
                    if is_safe:
                        return dx, dy
            
            # If no completely safe move is found, try to at least avoid direct collisions
            for dx, dy in possible_moves:
                new_x = entity.x + dx
                new_y = entity.y + dy
                if self.is_valid_move(new_x, new_y, entities, entity, next_positions):
                    return dx, dy
        else:
            # For other entities, just find any valid move
            for dx, dy in possible_moves:
                new_x = entity.x + dx
                new_y = entity.y + dy
                if self.is_valid_move(new_x, new_y, entities, entity, next_positions):
                    return dx, dy
        
        # If no valid move is found, stay in place
        return 0, 0

    def generate_data(self, num_samples=100, num_steps=10):
        data = []
        for _ in range(num_samples):
            scenario = []
            
            # Generate initial positions
            entities = self.generate_initial_positions()
            scenario.append(entities)
            
            # Generate subsequent steps
            for _ in range(num_steps - 1):
                new_entities = []
                next_positions = set()  # Track where entities will move next
                
                # First, copy all obstacles
                for entity in entities:
                    if not entity.is_movable:
                        new_entities.append(Entity(entity.x, entity.y, entity.type))
                        next_positions.add((entity.x, entity.y))

                # Move pedestrians first (they have priority in movement)
                pedestrian_moves = []
                for entity in entities:
                    if entity.type == 'pedestrian':
                        dx, dy = self.get_safe_move(entity, entities, next_positions)
                        new_x, new_y = entity.x + dx, entity.y + dy
                        next_positions.add((new_x, new_y))
                        pedestrian_moves.append((entity, dx, dy))

                # Then move other entities
                for entity in entities:
                    if entity.is_movable:
                        if entity.type == 'pedestrian':
                            # Apply pre-calculated pedestrian moves
                            for ped, dx, dy in pedestrian_moves:
                                if ped == entity:
                                    new_entity = Entity(entity.x, entity.y, entity.type)
                                    new_entity.move(dx, dy)
                                    new_entities.append(new_entity)
                                    break
                        else:
                            # Move other entities
                            dx, dy = self.get_safe_move(entity, entities, next_positions)
                            new_entity = Entity(entity.x, entity.y, entity.type)
                            new_entity.move(dx, dy)
                            new_entities.append(new_entity)
                            next_positions.add((new_entity.x, new_entity.y))
                
                scenario.append(new_entities)
                entities = new_entities
            
            data.append(scenario)
        return data

    def generate_initial_positions(self):
        """Generate non-overlapping initial positions with safe distances for pedestrians"""
        positions = set()
        entities = []

        # First place obstacles
        for _ in range(self.num_obstacles):
            while True:
                x, y = np.random.randint(0, self.grid_size, size=2)
                if (x, y) not in positions:
                    positions.add((x, y))
                    entities.append(Entity(x, y, 'obstacle'))
                    break

        # Then place vehicles
        for _ in range(self.num_vehicles):
            while True:
                x, y = np.random.randint(0, self.grid_size, size=2)
                if (x, y) not in positions:
                    positions.add((x, y))
                    entities.append(Entity(x, y, 'vehicle'))
                    break

        # Finally place pedestrians with minimum separation
        for _ in range(self.num_pedestrians):
            attempts = 0
            max_attempts = 100  # Prevent infinite loops
            
            while attempts < max_attempts:
                x, y = np.random.randint(0, self.grid_size, size=2)
                is_safe = True
                
                # Check distance from obstacles and other pedestrians
                for entity in entities:
                    if entity.type == 'obstacle' or entity.type == 'pedestrian':
                        if abs(entity.x - x) + abs(entity.y - y) <= 1:  # Manhattan distance
                            is_safe = False
                            break

                if (x, y) not in positions and is_safe:
                    positions.add((x, y))
                    entities.append(Entity(x, y, 'pedestrian'))
                    break
                    
                attempts += 1

            if attempts >= max_attempts:
                print(f"Warning: Could not place all pedestrians with safe distances after {max_attempts} attempts")
                break

        return entities
    
    def visualize_data(self, scenario, sample_id):
        # Create figure with adjusted size to accommodate legend
        fig = plt.figure(figsize=(8, 6))
        
        # Create gridspec to position the plot and legend
        gs = gridspec.GridSpec(1, 2, width_ratios=[4, 1])
        
        # Create main plot and legend axes
        ax = plt.subplot(gs[0])
        legend_ax = plt.subplot(gs[1])
        legend_ax.axis('off')  # Hide the legend axes frame
        
        collision_frames = []
        collision_locations = []
        
        # Pre-compute collision frames and locations
        for frame, entities in enumerate(scenario):
            collisions = detect_collision(entities)
            if collisions:
                collision_frames.append(frame)
                for entity1, entity2 in collisions:
                    collision_locations.append((entity1.x, entity1.y))

        def update(frame):
            ax.clear()
            legend_ax.clear()
            legend_ax.axis('off')
            
            entities = scenario[frame]
            
            # Create empty lists for legend handles and labels
            legend_elements = []
            
            # Draw entities and collect unique elements for legend
            vehicles = ax.scatter([], [], color='red', marker='o', label='Vehicle')
            pedestrians = ax.scatter([], [], color='blue', marker='^', label='Pedestrian')
            obstacles = ax.scatter([], [], color='green', marker='s', label='Obstacle')
            
            # Add to legend elements
            legend_elements.extend([vehicles, pedestrians, obstacles])
            
            # Draw actual entities
            for entity in entities:
                if entity.type == 'vehicle':
                    ax.scatter(entity.x, entity.y, color='red', marker='o')
                elif entity.type == 'pedestrian':
                    ax.scatter(entity.x, entity.y, color='blue', marker='^')
                else:  # obstacle
                    ax.scatter(entity.x, entity.y, color='green', marker='s')

            # Draw collision circles if this is a collision frame
            if frame in collision_frames:
                for x, y in collision_locations:
                    circle = Circle((x, y), radius=1.0, color='red', alpha=0.3)
                    ax.add_patch(circle)
                ax.text(0.02, 0.98, 'COLLISION!', transform=ax.transAxes, 
                       color='red', fontsize=12, verticalalignment='top')
                # Add collision indicator to legend
                collision_patch = plt.scatter([], [], c='red', alpha=0.3, marker='o', s=300, label='Collision Area')
                legend_elements.append(collision_patch)

            # Set up the main plot
            ax.set_xlim(-1, self.grid_size + 1)
            ax.set_ylim(-1, self.grid_size + 1)
            ax.set_title(f"Time Step {frame+1}")
            ax.grid(True)
            
            # Create legend in the separate axis
            legend_ax.legend(handles=legend_elements, 
                           labels=['Vehicle', 'Pedestrian', 'Obstacle', 'Collision Area'] if frame in collision_frames 
                           else ['Vehicle', 'Pedestrian', 'Obstacle'],
                           loc='center left',
                           bbox_to_anchor=(0, 0.5))
            
            # Adjust layout to prevent overlap
            plt.tight_layout()

        ani = FuncAnimation(fig, update, frames=len(scenario), interval=500, repeat=False)

        # Create the animations folder if it doesn't exist
        os.makedirs('animations', exist_ok=True)

        # Save the animation as an HTML file
        html_path = f'animations/scenario_{sample_id}_nocollision.html'
        writer = HTMLWriter(fps=2)
        ani.save(html_path, writer=writer)

        plt.close(fig)
        return html_path

    def analyze_collisions(self, scenario):
        collision_report = []
        for timestep, entities in enumerate(scenario):
            collisions = detect_collision(entities)
            if collisions:
                for entity1, entity2 in collisions:
                    collision_report.append({
                        'timestep': timestep + 1,
                        'location': (entity1.x, entity1.y),
                        'entities': (entity1.type, entity2.type)
                    })
        return collision_report

In [6]:
generator = SyntheticDataGenerator(grid_size=20, num_vehicles=5, num_pedestrians=5, num_obstacles=5)
data = generator.generate_data(num_samples=1, num_steps=100)

for i, scenario in enumerate(data):
    print(f"\nScenario {i+1}:")
    html_path = generator.visualize_data(scenario, sample_id=i+1)
    display(IFrame(src=html_path, width=800, height=800))

    # Get and display detailed collision report
    collision_report = generator.analyze_collisions(scenario)
    if collision_report:
        print("\nCollision Report:")
        for collision in collision_report:
            print(f"Time Step {collision['timestep']}: "
                  f"{collision['entities'][0].capitalize()} collided with "
                  f"{collision['entities'][1]} at position {collision['location']}")
    else:
        print("No collisions detected in this scenario")
    print("-" * 50)


Scenario 1:



Collision Report:
Time Step 3: Vehicle collided with pedestrian at position (8, 18)
Time Step 9: Obstacle collided with vehicle at position (14, 16)
Time Step 12: Obstacle collided with vehicle at position (15, 16)
Time Step 18: Obstacle collided with vehicle at position (14, 16)
Time Step 23: Vehicle collided with vehicle at position (12, 8)
Time Step 24: Vehicle collided with pedestrian at position (13, 17)
Time Step 26: Vehicle collided with pedestrian at position (13, 17)
Time Step 32: Obstacle collided with vehicle at position (15, 16)
Time Step 36: Vehicle collided with pedestrian at position (12, 8)
--------------------------------------------------
